### ETL process

#### Extract
* Import csv file into Pandas DataFrame.
* Import partners csvs to send to PostgresSQL database

#### Transform
* Drop columns not required..
* Add new columns and calculate their values.
* Drop columns no longer required.
* Get zip code
* Drop columns no longer needed.
* Group data by zip code and get count of airbnbs in area and average price per night
* Delete missing data and typos.


#### Load

* Export all 3 dataframes (zipcode_df, zillow_df, complaints_df) to PostgreSQL database.

In [119]:
# Import all modules
import pandas as pd
import geopy
from sqlalchemy import create_engine

#### pull data from air_listings_expanded_chicago.csv and air_listings_chicago.csv

In [3]:

airbnb_exp_file_path = "../Resources/air_listings_expanded_chicago.csv"
airbnb_exp_df = pd.read_csv(airbnb_exp_file_path)
airbnb_exp_df.head()

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2384,https://www.airbnb.com/rooms/2384,20201024055553,2020-10-24,"Hyde Park - Walk to UChicago, 10 min to McCormick",Visiting The University of Chicago or attendin...,The apartment is less than one block from beau...,https://a0.muscache.com/pictures/acf6b3c0-47f2...,2613,https://www.airbnb.com/users/show/2613,...,10.0,10.0,10.0,R17000015609,f,1,0,1,0,2.55
1,4505,https://www.airbnb.com/rooms/4505,20201024055553,2020-10-24,394 Great Reviews. 127 y/o House. 40 yds to tr...,We have been closed for quite a while and will...,NaN,https://a0.muscache.com/pictures/10628083/3543...,5775,https://www.airbnb.com/users/show/5775,...,10.0,10.0,9.0,R19000037538,t,1,1,0,0,2.79
2,7126,https://www.airbnb.com/rooms/7126,20201024055553,2020-10-24,Tiny Studio Apartment 94 Walk Score,A very small studio in a wonderful neighborhoo...,"Ukrainian Village was just named ""Hottest Neig...",https://a0.muscache.com/pictures/51073/16c81c7...,17928,https://www.airbnb.com/users/show/17928,...,10.0,10.0,10.0,City registration pending,f,1,1,0,0,2.80
3,9811,https://www.airbnb.com/rooms/9811,20201024055553,2020-10-24,Barbara's Hideaway - Old Town,One-bedroom hideaway tucked into Old Town step...,Chicago’s Old Town neighborhood is squeezed be...,https://a0.muscache.com/pictures/37df1935-855c...,33004,https://www.airbnb.com/users/show/33004,...,9.0,10.0,9.0,2079260,f,9,9,0,0,0.65
4,10610,https://www.airbnb.com/rooms/10610,20201024055553,2020-10-24,3 Comforts of Cooperative Living,The condo is the 2nd floor in a lovely 1912 3-...,It's a 10 minute walk from the lakefront bike ...,https://a0.muscache.com/pictures/51944609/8d76...,2140,https://www.airbnb.com/users/show/2140,...,9.0,10.0,9.0,City registration pending,f,2,0,2,0,0.62


In [42]:
airbnb_file_path = "../Resources/air listings_chicago.csv"
airbnb_df = pd.read_csv(airbnb_file_path)
airbnb_df.insert(8,"zipcode", '')
airbnb_df

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,zipcode,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2384,"Hyde Park - Walk to UChicago, 10 min to McCormick",2613,Rebecca,NaN,Hyde Park,41.78790,-87.587800,,Private room,65,2,180,2020-10-03,2.55,1,359
1,4505,394 Great Reviews. 127 y/o House. 40 yds to tr...,5775,Craig & Kathleen,NaN,South Lawndale,41.85495,-87.696960,,Entire home/apt,105,2,395,2020-07-14,2.79,1,187
2,7126,Tiny Studio Apartment 94 Walk Score,17928,Sarah,NaN,West Town,41.90289,-87.681820,,Entire home/apt,60,2,385,2020-09-14,2.80,1,347
3,9811,Barbara's Hideaway - Old Town,33004,At Home Inn,NaN,Lincoln Park,41.91769,-87.637880,,Entire home/apt,66,4,52,2020-09-29,0.65,9,310
4,10610,3 Comforts of Cooperative Living,2140,Lois,NaN,Hyde Park,41.79612,-87.592610,,Private room,22,1,45,2020-09-15,0.62,2,201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6290,46010835,Cozy Two Bedroom,175692579,Deborah,NaN,Douglas,41.83283,-87.618630,,Entire home/apt,110,3,0,NaN,NaN,2,355
6291,46020831,Live + Work + Stay + Easy | 1BR in Chicago,359234447,Kia,NaN,Near North Side,41.89111,-87.622050,,Entire home/apt,116,33,0,NaN,NaN,54,277
6292,46022331,West Ridge private room,156571992,Bogdan,NaN,West Ridge,42.00124,-87.681430,,Private room,21,1,0,NaN,NaN,1,263
6293,46022730,A Family Get Away -All the Amenities to Stay IN,291691868,Michael And Angel,NaN,West Town,41.90518,-87.664358,,Entire home/apt,306,2,0,NaN,NaN,1,83


#### defined function for finding the zipcode from a set of latitude and longitude values
runtime = 1h 14m


In [43]:


def get_zipcode(lat_field, lon_field, df, index):
    try:
        geolocator = geopy.Nominatim(user_agent='air_bnb_review', timeout = 10)
        location = geolocator.reverse((lat_field, lon_field))
        df.at[index, "zipcode"] = location.raw["address"]["postcode"]
    except (KeyError):
        pass
    return

#### call to function to get data  and export to dataframe

In [44]:

for index, row in airbnb_df.iterrows():
    get_zipcode(row["latitude"], row["longitude"], airbnb_df, index)
    


In [46]:
airbnb_df[airbnb_df["zipcode"] ==""]

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,zipcode,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
130,1516154,Best Location Millennium Park Extended Stay POOL,3965428,Rob,NaN,Loop,41.88642,-87.61695,,Entire home/apt,60,31,18,2015-08-16,0.21,74,365
207,2661465,GREAT LOCATION SPECTACULAR VIEWS + BALCONY,3965428,Rob,NaN,Loop,41.88618,-87.61888,,Entire home/apt,120,31,25,2018-06-29,0.32,74,365
333,4503332,Near The Loop and McCormick Place,20059524,(Email hidden by Airbnb),NaN,Armour Square,41.85586,-87.63569,,Entire home/apt,100,1,86,2020-03-15,1.31,3,87
432,6156931,★MUST LOCATION 2 BEDROOMS + 2 BATH FANTASTIC V...,3965428,Rob,NaN,Loop,41.88626,-87.61632,,Entire home/apt,175,60,14,2017-08-31,0.21,74,364
446,6375348,Convenient Quiet Oasis near Downtown and Beach,16050215,Terryon,NaN,Douglas,41.83240,-87.61058,,Private room,50,3,184,2020-10-11,2.81,1,250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6233,45880617,Live + Work + Stay + Easy | 1BR in Chicago,359234447,Kia,NaN,Near North Side,41.89111,-87.62205,,Entire home/apt,128,33,0,NaN,NaN,54,327
6244,45909330,Live + Work + Stay + Easy | 2BR in Chicago,359234447,Kia,NaN,Near North Side,41.89111,-87.62205,,Entire home/apt,160,33,0,NaN,NaN,54,331
6266,45973926,Live + Work + Stay + Easy | 2BR in Chicago,359234447,Kia,NaN,Near North Side,41.89111,-87.62205,,Entire home/apt,163,33,0,NaN,NaN,54,267
6269,45976728,Live + Work + Stay + Easy | 3BR in Chicago,359234447,Kia,NaN,Near North Side,41.89111,-87.62205,,Entire home/apt,229,33,0,NaN,NaN,54,304


## clean data to find only the relevent zipcodes

In [77]:
airbnb_full_df = airbnb_df[airbnb_df["zipcode"] != ""]
try:
    airbnb_full_df = pd.to_numeric(airbnb_full_df["zipcode"])
    airbnb_full_df = airbnb_full_df[airbnb_full_df["zipcode"] <=70000]
    airbnb_full_df = airbnb_full_df[airbnb_full_df["zipcode"] >=60000]
except(ValueError):
    pass

In [137]:
grouped_air_df = airbnb_full_df.groupby("zipcode").agg(count=('id', 'size'), mean=('price', 'mean')).reset_index()
pd.set_option('display.max_rows', 100)
grouped_air_df = grouped_air_df.rename(columns ={"zipcode": "zip_code", "count": "air_bnb_count", "mean": "avg_air_bnb_cost"})
grouped_air_df

,zip_code,air_bnb_count,avg_air_bnb_cost
0,2300,1,45.000000
1,46394,1,50.000000
2,60018,1,39.000000
3,60171,1,107.000000
4,60302,37,143.297297
5,60304,1,350.000000
6,60402,3,69.666667
7,60453,1,35.000000
8,606,8,85.750000
9,60601,111,456.423423


In [138]:
filtrered_df = grouped_air_df[grouped_air_df['zip_code'].str.len() == 5]
filtered_df = filtrered_df[filtrered_df['zip_code'] != "46394"]
filtered_df = filtered_df[filtrered_df['zip_code'] != "90615"]
air_bnb_df = filtered_df.set_index('zip_code')
air_bnb_df


C:\Users\staff\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


,air_bnb_count,avg_air_bnb_cost
zip_code,,
60018,1,39.000000
60171,1,107.000000
60302,37,143.297297
60304,1,350.000000
60402,3,69.666667
60453,1,35.000000
60601,111,456.423423
60602,24,153.125000
60603,7,137.857143


# Load

Create engine connecting to Postgress SQL and send dataframes 

In [90]:
connection_string = "postgres:R3ds0x916!@localhost:5432/chicago_db"
engine = create_engine(f'postgresql://{connection_string}')

In [142]:
# test to see if all tables are found in the postgresSQl Database
engine.table_names()

['air_bnb', 'zillow', 'complaints']

#### Export air_bnb_df to Sql

In [139]:
air_bnb_df.to_sql(name='air_bnb', con=engine, if_exists='append', index=True)

In [123]:
#
zillow_df = pd.read_csv("../Abraham/zillow_zip.csv")
zillow_df = zillow_df[["zip_code", "avg_price_per_sq_ft", "avg_price"]]
zillow_df = zillow_df.rename(columns={"avg_price_per_sq_ft" : "zillow_avg_price_per_sq_ft", "avg_price": "zillow_avg_price"})
zillow_df.head()

,zip_code,zillow_avg_price_per_sq_ft,zillow_avg_price
0,60608,225.082960,286108.757576
1,60614,456.706207,577682.778689
2,60615,140.517034,181651.077778
3,60617,77.808295,88679.166667
4,60618,295.810959,417163.265306


#### Export zillow_df to Sql

In [140]:
zillow_df.to_sql(name='zillow', con=engine, if_exists='append', index=False)

In [107]:
complaints_df = pd.read_csv("../Veronica/Noise_complaint_counts_by_zip.csv")
complaints_df.head()

,zip_code,num_complaints
0,60601,7
1,60602,4
2,60603,4
3,60604,1
4,60605,54


#### Export complaints_df to Sql

In [141]:
complaints_df.to_sql(name='complaints', con=engine, if_exists='append', index=False)